# NWB-Adapter


DataJoint 0.12 introduced new features that enabling handling of using specialized file formats in conjunction with DataJoint.
This notebook demostrates using Neurodata Without Borders files.

In [1]:
# Author and date
import datetime, getpass
print(getpass.getuser(), datetime.datetime.today())

import datajoint as dj
print('DataJoint Version', dj.__version__)

dimitri 2019-11-06 08:06:36.148912
DataJoint Version 0.12.1


In [1]:
%matplotlib inline
import os
import datajoint as dj
import pynwb
from pynwb import NWBFile, NWBHDF5IO
from datetime import datetime
from dateutil.tz import tzlocal
import json
import numpy as np
import pathlib

In [2]:
# enable advanced features, which are disabled by default 
os.environ['DJ_SUPPORT_ADAPTED_TYPES'] = 'TRUE'
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'
dj.config['enable_python_native_blobs'] = True

In [3]:
dj.__version__

'0.12.1'

In [4]:
dj.config['database.host'] = 'workshop-db.datajoint.io'
dj.conn().connect()

Please enter DataJoint username: thinh
Please enter DataJoint password: ········
Connecting thinh@workshop-db.datajoint.io:3306


In [30]:
import warnings
warnings.filterwarnings('ignore')

# Objective

This notebook presents a specific example usecase of incorporating ***NWB*** objects into DataJoint using the new ***dj.AttributeAdapter*** feature. 

We wish to create a table storing NWB object, with attribute type of ***filepath***. The idea is to generate NWB files, one for each session, that can be access from the file system, hence type ***filepath***, and can also be fetched and worked with as part of DatJoint pipeline. This can be accomplished with the new ***dj.AttributeAdapter*** feature.

As prerequisite, readers should be familiar with the concept and operation of ***dj.AttributeAdapter***, a review can be found [here](./Adapted-Types.ipynb).

## Step 0 - Create a ***store*** for the filepath

In [6]:
exported_nwb_dir = 'C:/Users/thinh/Documents/nwb_store'

In [7]:
dj.config['stores'] = {
    'nwbstore': {'protocol': 'file',
                 'stage': exported_nwb_dir,
                 'location': exported_nwb_dir}}

## Step 1 - Create a DataJoint AttributeAdapter for NWB object

Basically we will need to define an object inhereted from `dj.AttributeAdapter` and instantiated with a variable name ***nwb_obj***

In [8]:
class NWBAdapter(dj.AttributeAdapter):
    attribute_type = 'filepath@nwbstore'
    
    def put(self, nwb):
        save_file_name = ''.join([nwb.identifier, '.nwb'])
        with NWBHDF5IO(os.path.join(exported_nwb_dir, save_file_name), mode='w') as io:
            io.write(nwb)
            print(f'Write NWB 2.0 file: {save_file_name}')
        return os.path.join(exported_nwb_dir, save_file_name)
        
    def get(self, path):
        return NWBHDF5IO(path, mode='r').read()

#### Instantiate for use as a datajoint type

In [9]:
nwb_obj = NWBAdapter()

## Step 2 - Create a new schema ***export*** and NWB table

This ***NWB*** table specifies a primary key of `experiment.Session`, designed to store one NWB object (or NWBFile) per session

In [10]:
schema = dj.schema('djneuro_nwb_adapter')

In [11]:
@schema
class NWB(dj.Manual):
    definition = """
    nwb_id: int
    ---
    nwb: <nwb_obj> 
    """

Note that the table definition above set the ***nwb*** attribute to be of type ***< nwb_obj >***. 

Hence the reason for defining ***nwbfile*** as an instant of ***NWBAdapter*** - see Step 1

## Step 3 - Build an NWBFile

Here, we build a very simple NWB object using the `pynwb` package, for the sake of demonstration

In [31]:
# -- create NWB 
nwb = NWBFile(identifier='nwb_01',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [13]:
nwb


root <class 'pynwb.file.NWBFile'>
Fields:
  experimenter: John Smith
  file_create_date: [datetime.datetime(2019, 11, 4, 11, 1, 3, 203603, tzinfo=tzlocal())]
  identifier: nwb_01
  session_start_time: 2019-10-20 00:00:00-05:00
  subject: subject <class 'pynwb.file.Subject'>
  timestamps_reference_time: 2019-10-20 00:00:00-05:00

## Step 4 -  Insert to the ***NWB*** table

In [14]:
NWB()

nwb_id,nwb


In [15]:
NWB.insert1({'nwb_id': 0, 'nwb': nwb})

Write NWB 2.0 file: nwb_01.nwb


In [16]:
NWB()

nwb_id,nwb
0,=BLOB=


### Now, fetch that NWB file back

In [17]:
fetched_nwb = (NWB & 'nwb_id=0').fetch1('nwb')

In [18]:
fetched_nwb


root <class 'pynwb.file.NWBFile'>
Fields:
  experimenter: ['John Smith']
  file_create_date: [datetime.datetime(2019, 11, 4, 11, 1, 3, 203603, tzinfo=tzoffset(None, -21600))]
  identifier: nwb_01
  session_start_time: 2019-10-20 00:00:00-05:00
  subject: subject <class 'pynwb.file.Subject'>
  timestamps_reference_time: 2019-10-20 00:00:00-05:00

## This concludes the basic showcase of using `dj.AttributeAdapter` to work with `NWB` objects

Continue further to see more examples, but the core usage is demonstrated above

In [32]:
# -- create NWB 
nwb2 = NWBFile(identifier='nwb_02',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb2.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [33]:
# -- create NWB 
nwb3 = NWBFile(identifier='nwb_03',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb3.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [25]:
NWB.insert([{'nwb_id': 2, 'nwb': nwb2},
            {'nwb_id': 3, 'nwb': nwb3}])

Write NWB 2.0 file: nwb_02.nwb
Write NWB 2.0 file: nwb_03.nwb


In [26]:
NWB()

nwb_id,nwb
3,=BLOB=
2,=BLOB=
0,=BLOB=


In [27]:
fetch_nwb3 = (NWB & 'nwb_id=3').fetch1('nwb')

In [28]:
fetch_nwb3


root <class 'pynwb.file.NWBFile'>
Fields:
  experimenter: ['John Smith']
  file_create_date: [datetime.datetime(2019, 11, 4, 11, 5, 5, 538754, tzinfo=tzoffset(None, -21600))]
  identifier: nwb_03
  session_start_time: 2019-10-20 00:00:00-05:00
  subject: subject <class 'pynwb.file.Subject'>
  timestamps_reference_time: 2019-10-20 00:00:00-05:00

### Let's also look at the directory where all the NWB files are generated (configured in the `nwbstore`)

In [29]:
os.listdir(exported_nwb_dir)

['nwb_01.nwb', 'nwb_02.nwb', 'nwb_03.nwb']